In [1]:
import joblib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import uuid

2026-01-16 15:58:15.587291: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 15:58:15.604574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768575495.614663   58718 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768575495.618026   58718 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768575495.626689   58718 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
def display_array_as_image(array, title="Preprocessed Image"):
    reshaped_array = array.reshape(28, 28)

    plt.imshow(reshaped_array, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def class_to_char_emnist_bymerge(class_idx):
    """
    Convert EMNIST bymerge class index to character.
    The mapping is based on the dataset's official character set.
    """
    # EMNIST bymerge mapping (A-Z, then a-z, then 0-9) with merged cases
    # This is the official mapping from the EMNIST documentation
    mapping = {
        0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
        10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I',
        19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R',
        28: 'S', 29: 'T', 30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z',
        36: 'a', 37: 'b', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'n', 44: 'q',
        45: 'r', 46: 't'
    }
    
    # Check if the index is valid
    if class_idx in mapping:
        return mapping[class_idx]
    else:
        return "?" # Return a placeholder for an unknown index

def class_to_char(class_idx):
    """Convert class index to character (A-Z: 0-25, 0-9: 26-35)"""
    if class_idx < 10:
        return str(class_idx)
    else:
        return chr(class_idx + 55)
        
def class_to_char_alpha(class_idx):
    """Convert class index to character (A-Z: 0-25, 0-9: 26-35)"""
    return chr(class_idx+65)

In [5]:
model = joblib.load('models/sklearn_logistic_regression/model.joblib')

In [7]:
base_dir_cnn = "models/cnn/"
alpha = "alpha_cnn.h5"
alpha2 = "alpha_recognition_cnn2.h5"
alpha3 = "alpha_recognition_cnn3.h5"
alpha4 = "alpha_test_50_cnn.keras"
alphanum = "alphanum_cnn.h5"
alphanum2 = "alphanum_recognition_cnn.h5"
alphanum3 = "alphanum_regularized_cnn.h5"
alphanum4 = "alpha_recognition_cnn2.h5"
emnist = "emnist_cnn.h5"
emnist_az = ""
emnist_az_reduced = "eminst_a-z_reduced_cnn.h5"


cnn_model = tf.keras.models.load_model(base_dir_cnn + alpha4)

2026-01-16 15:58:20.226555: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
from PIL import Image, ImageOps

def preprocess_image_for_cnn(image_path):
    """
    Loads an image from a given path, preprocesses it, and converts it
    to a NumPy array suitable for CNN model prediction.
    """
    try:
        img = Image.open(image_path)
        img = img.convert('L')
        img = img.resize((28, 28), Image.LANCZOS)
        img_array = np.array(img, dtype=np.float32) / 255.0
        # Invert the pixel values to match the dataset (white on black)
        img_array = 1.0 - img_array
        
        # --- Display the 2D array as an image (if function exists) ---
        #display_array_as_image(img_array) 
        
        # Reshape for CNN: (1, 28, 28, 1) instead of flattening
        cnn_array = img_array.reshape(1, 28, 28, 1)
        return cnn_array
        
    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def preprocess_image_for_cnn2(image_path):
    try:
        img = Image.open(image_path)
        img = img.convert('L')
        
        img_np = np.array(img, dtype=np.uint8)

        _, img_binary = cv2.threshold(img_np, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        kernel = np.ones((3, 3), np.uint8)
        img_dilated_binary = cv2.dilate(img_binary, kernel, iterations=3)
        
        img_pil_dilated = Image.fromarray(img_dilated_binary)

        img = img_pil_dilated.resize((28, 28), Image.LANCZOS)
        img_array = np.array(img, dtype=np.float32) / 255.0
        
        display_array_as_image(img_array) 
        
        cnn_array = img_array.reshape(1, 28, 28, 1)
        return cnn_array
        
    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def preprocess_image_for_cnn3(image_path):
    """
    Loads an image from a given path, preprocesses it, and converts it
    to a NumPy array suitable for CNN model prediction.
    Applies EMNIST-specific transformations: 90° clockwise rotation and vertical flip.
    """
    try:
        img = Image.open(image_path)
        img = img.convert('L')
        img = img.resize((28, 28), Image.LANCZOS)

        """
        img_array1 = np.array(img, dtype=np.float32) / 255.0
        # Invert the pixel values to match the dataset (white on black)
        img_array1 = 1.0 - img_array1
        
        # --- Display the 2D array as an image (if function exists) ---
        display_array_as_image(img_array1)
        """
        
        # Apply EMNIST transformations
        img = img.rotate(90, expand=False)  # Rotate 90° clockwise (-90)
        img = img.transpose(Image.FLIP_TOP_BOTTOM)  # Flip vertically
        
        img_array = np.array(img, dtype=np.float32) / 255.0
        # Invert the pixel values to match the dataset (white on black)
        img_array = 1.0 - img_array
        
        # --- Display the 2D array as an image (if function exists) ---
        # display_array_as_image(img_array) 
        
        # Reshape for CNN: (1, 28, 28, 1) instead of flattening
        cnn_array = img_array.reshape(1, 28, 28, 1)
        return cnn_array
        
    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


### WITHOUT FALSE POSITIVES

In [23]:

def benchmark_model(test_data_directory="testing_data/benchmark1"):
    """
    Benchmark the model by testing it on sorted data and calculating accuracy
    
    Args:
        test_data_directory: Directory containing sorted character folders
    """
    
    if not os.path.exists(test_data_directory):
        print(f"Error: Test data directory '{test_data_directory}' not found!")
        return
    
    # Get all character folders
    character_folders = [f for f in os.listdir(test_data_directory) 
                        if os.path.isdir(os.path.join(test_data_directory, f))]
    
    if not character_folders:
        print("No character folders found in test directory!")
        return
    
    character_folders.sort()
    
    total_correct = 0
    total_images = 0
    overall_results = {}
    
    print(f"Benchmarking model on {len(character_folders)} character classes...")
    print("="*60)
    
    for true_char in character_folders:
        folder_path = os.path.join(test_data_directory, true_char)
        
        # Get all image files in this character folder
        image_files = []
        
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                image_files.append(filename)
        
        if not image_files:
            print(f"No images found in folder '{true_char}'")
            continue
        
        print(f"\nTesting character '{true_char}' ({len(image_files)} images):")
        
        # Dictionary to count predictions for this character
        prediction_counts = {}
        correct_predictions = 0
        
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_cnn(image_path)
            
            if preprocessed_image is not None:
                # Get prediction
                prediction_proba = cnn_model.predict(preprocessed_image, verbose=0)
                predicted_class_id = np.argmax(prediction_proba[0])
                predicted_char = class_to_char(predicted_class_id)
                confidence = np.max(prediction_proba[0])
                
                # Count predictions
                if predicted_char in prediction_counts:
                    prediction_counts[predicted_char] += 1
                else:
                    prediction_counts[predicted_char] = 1
                
                # Check if prediction is correct
                if predicted_char == true_char:
                    correct_predictions += 1
                
                total_images += 1
        
        # Calculate accuracy for this character
        folder_accuracy = (correct_predictions / len(image_files)) * 100 if image_files else 0
        total_correct += correct_predictions
        
        # Display results for this character
        print(f"Predictions for '{true_char}':")
        sorted_predictions = sorted(prediction_counts.items(), key=lambda x: x[1], reverse=True)
        for pred_char, count in sorted_predictions:
            percentage = (count / len(image_files)) * 100
            marker = "✓" if pred_char == true_char else "✗"
            print(f"  {marker} {pred_char}: {count} times ({percentage:.1f}%)")
        
        print(f"Accuracy for '{true_char}': {correct_predictions}/{len(image_files)} ({folder_accuracy:.1f}%)")
        
        # Store overall results
        overall_results[true_char] = {
            'total_images': len(image_files),
            'correct_predictions': correct_predictions,
            'accuracy': folder_accuracy,
            'predictions': prediction_counts
        }
    
    # Calculate and display overall accuracy
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%")
    
    # Display per-character summary
    print("\nPer-character accuracy:")
    for char, results in overall_results.items():
        print(f"  {char}: {results['accuracy']:.1f}% ({results['correct_predictions']}/{results['total_images']})")
    
    # Find best and worst performing characters
    if overall_results:
        best_char = max(overall_results.items(), key=lambda x: x[1]['accuracy'])
        worst_char = min(overall_results.items(), key=lambda x: x[1]['accuracy'])
        
        print(f"\nBest performing: '{best_char[0]}' ({best_char[1]['accuracy']:.1f}%)")
        print(f"Worst performing: '{worst_char[0]}' ({worst_char[1]['accuracy']:.1f}%)")
    
    return overall_results

### FALSE POSITIVES

In [27]:
def benchmark_model_fp(test_data_directory="testing_data/benchmark1"):
    """
    Benchmark the model by testing it on sorted data and calculating accuracy and
    false positives per class.
    
    Args:
        test_data_directory: Directory containing sorted character folders
    """
    import os
    import numpy as np
    # NOTE: cnn_model, preprocess_image_for_cnn, and class_to_char must be defined globally
    # or passed as arguments for this function to run.
    
    if not os.path.exists(test_data_directory):
        print(f"Error: Test data directory '{test_data_directory}' not found!")
        return
    
    # Get all character folders
    character_folders = [f for f in os.listdir(test_data_directory) 
                         if os.path.isdir(os.path.join(test_data_directory, f))]
    
    if not character_folders:
        print("No character folders found in test directory!")
        return
    
    character_folders.sort()
    
    total_correct = 0
    total_images = 0
    overall_results = {}
    
    # Initialize a dictionary to track False Positives across ALL classes
    false_positives = {char: 0 for char in character_folders}
    
    print(f"Benchmarking model on {len(character_folders)} character classes...")
    print("="*60)
    
    for true_char in character_folders:
        folder_path = os.path.join(test_data_directory, true_char)
        
        # Get all image files in this character folder
        image_files = []
        
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                image_files.append(filename)
        
        if not image_files:
            print(f"No images found in folder '{true_char}'")
            continue
        
        print(f"\nTesting character '{true_char}' ({len(image_files)} images):")
        
        # Dictionary to count predictions for this character
        prediction_counts = {}
        correct_predictions = 0
        
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_cnn(image_path)
            
            if preprocessed_image is not None:
                # Get prediction
                prediction_proba = cnn_model.predict(preprocessed_image, verbose=0)
                predicted_class_id = np.argmax(prediction_proba[0])
                predicted_char = class_to_char(predicted_class_id)
                confidence = np.max(prediction_proba[0])
                
                # Count predictions
                if predicted_char in prediction_counts:
                    prediction_counts[predicted_char] += 1
                else:
                    prediction_counts[predicted_char] = 1
                
                # Check if prediction is correct
                if predicted_char == true_char:
                    correct_predictions += 1
                else:
                    # Increment False Positive count for the PREDICTED character
                    false_positives[predicted_char] += 1
                
                total_images += 1
        
        # Calculate accuracy for this character
        folder_accuracy = (correct_predictions / len(image_files)) * 100 if image_files else 0
        total_correct += correct_predictions
        
        # Display results for this character
        print(f"Predictions for '{true_char}':")
        sorted_predictions = sorted(prediction_counts.items(), key=lambda x: x[1], reverse=True)
        for pred_char, count in sorted_predictions:
            percentage = (count / len(image_files)) * 100
            marker = "✓" if pred_char == true_char else "✗"
            print(f"  {marker} {pred_char}: {count} times ({percentage:.1f}%)")
        
        print(f"Accuracy for '{true_char}': {correct_predictions}/{len(image_files)} ({folder_accuracy:.1f}%)")
        
        # Store overall results
        overall_results[true_char] = {
            'total_images': len(image_files),
            'correct_predictions': correct_predictions,
            'accuracy': folder_accuracy,
            'predictions': prediction_counts
        }
    
    # Calculate and display overall accuracy
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%")
    
    # Display per-character summary
    print("\nPer-character summary:")
    for char in character_folders:
        results = overall_results.get(char, {'accuracy': 0.0, 'correct_predictions': 0, 'total_images': 0})
        fp_count = false_positives.get(char, 0)
        
        print(f"  {char}: Acc={results['accuracy']:.1f}% ({results['correct_predictions']}/{results['total_images']}), FP={fp_count}")
        
        # Store False Positives in the results structure
        overall_results[char]['false_positives'] = fp_count
        
    
    # Find best and worst performing characters (unchanged)
    if overall_results:
        best_char = max(overall_results.items(), key=lambda x: x[1]['accuracy'])
        worst_char = min(overall_results.items(), key=lambda x: x[1]['accuracy'])
        
        print(f"\nBest performing: '{best_char[0]}' ({best_char[1]['accuracy']:.1f}%)")
        print(f"Worst performing: '{worst_char[0]}' ({worst_char[1]['accuracy']:.1f}%)")
    
    return overall_results

### FALSE POSITIVES WITH DETAILS

In [13]:
import cv2

def benchmark_model_fp_details(test_data_directory="testing_data/benchmark1"):
    """
    Benchmark the model by testing it on sorted data and calculating accuracy and
    detailed false positives per class.
    
    Args:
        test_data_directory: Directory containing sorted character folders
    """
    import os
    import numpy as np
    # NOTE: cnn_model, preprocess_image_for_cnn, and class_to_char must be defined globally
    # or passed as arguments for this function to run.
    
    if not os.path.exists(test_data_directory):
        print(f"Error: Test data directory '{test_data_directory}' not found!")
        return
    
    # Get all character folders
    character_folders = [f for f in os.listdir(test_data_directory) 
                         if os.path.isdir(os.path.join(test_data_directory, f))]
    
    if not character_folders:
        print("No character folders found in test directory!")
        return
    
    character_folders.sort()
    
    total_correct = 0
    total_images = 0
    overall_results = {}
    
    # NEW: Initialize a dictionary to track False Positives in detail.
    # Structure: {predicted_char: {true_char: count, ...}, ...}
    detailed_false_positives = {char: {} for char in character_folders}
    
    print(f"Benchmarking model on {len(character_folders)} character classes...")
    print("="*60)
    
    for true_char in character_folders:
        folder_path = os.path.join(test_data_directory, true_char)
        
        # Get all image files in this character folder
        image_files = []
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                image_files.append(filename)
        
        if not image_files:
            print(f"No images found in folder '{true_char}'")
            continue
        
        print(f"\nTesting character '{true_char}' ({len(image_files)} images):")
        
        # Dictionary to count predictions for this character (True Positive + False Negatives)
        prediction_counts = {}
        correct_predictions = 0
        
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_cnn3(image_path)
            
            if preprocessed_image is not None:
                # Get prediction
                prediction_proba = cnn_model.predict(preprocessed_image, verbose=0)
                predicted_class_id = np.argmax(prediction_proba[0])
                predicted_char = class_to_char(predicted_class_id)
                confidence = np.max(prediction_proba[0])
                
                # Count predictions
                prediction_counts[predicted_char] = prediction_counts.get(predicted_char, 0) + 1
                
                # Check if prediction is correct
                if predicted_char == true_char:
                    correct_predictions += 1
                else:
                    # NEW: Update the detailed False Positive tracker
                    # The model predicted 'predicted_char' when the true label was 'true_char'
                    if true_char in detailed_false_positives[predicted_char]:
                        detailed_false_positives[predicted_char][true_char] += 1
                    else:
                        detailed_false_positives[predicted_char][true_char] = 1
                
                total_images += 1
        
        # Calculate accuracy for this character
        folder_accuracy = (correct_predictions / len(image_files)) * 100 if image_files else 0
        total_correct += correct_predictions
        
        # Display results for this character
        print(f"Predictions for '{true_char}':")
        sorted_predictions = sorted(prediction_counts.items(), key=lambda x: x[1], reverse=True)
        for pred_char, count in sorted_predictions:
            percentage = (count / len(image_files)) * 100
            marker = "✓" if pred_char == true_char else "✗"
            print(f"  {marker} {pred_char}: {count} times ({percentage:.1f}%)")
        
        print(f"Accuracy for '{true_char}': {correct_predictions}/{len(image_files)} ({folder_accuracy:.1f}%)")
        
        # Store results
        overall_results[true_char] = {
            'total_images': len(image_files),
            'correct_predictions': correct_predictions,
            'accuracy': folder_accuracy,
            'predictions': prediction_counts
        }
    
    # Calculate and display overall accuracy
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%")
    
    # Display per-character summary including detailed FPs
    print("\nPer-character summary (Accuracy & Confusion):")
    for char in character_folders:
        results = overall_results.get(char, {'accuracy': 0.0, 'correct_predictions': 0, 'total_images': 0})
        fp_details = detailed_false_positives.get(char, {})
        
        # Calculate total False Positives for this character (how many times it was predicted incorrectly)
        total_fp = sum(fp_details.values())
        
        # Format the detailed FP string (e.g., "4-Y, 2-U")
        fp_list = []
        if total_fp > 0:
            # Sort by count descending for readability
            sorted_fp = sorted(fp_details.items(), key=lambda x: x[1], reverse=True)
            fp_list = [f"{count}-{true_char}" for true_char, count in sorted_fp]
            
        fp_str = ", ".join(fp_list) if fp_list else "None"
        
        print(f"  {char}: Acc={results['accuracy']:.1f}% ({results['correct_predictions']}/{results['total_images']})")
        print(f"      FP ({total_fp}): {fp_str}")
        
        # Store False Positives in the results structure
        overall_results[char]['false_positives_total'] = total_fp
        overall_results[char]['false_positives_details'] = fp_details
    
    # Find best and worst performing characters (unchanged)
    if overall_results:
        best_char = max(overall_results.items(), key=lambda x: x[1]['accuracy'])
        worst_char = min(overall_results.items(), key=lambda x: x[1]['accuracy'])
        
        print(f"\nBest performing: '{best_char[0]}' ({best_char[1]['accuracy']:.1f}%)")
        print(f"Worst performing: '{worst_char[0]}' ({worst_char[1]['accuracy']:.1f}%)")
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%") # <-- THIS IS THE FINAL METRIC
    
    return overall_results

### EMNIST

In [13]:
import cv2

def benchmark_model_emnist(test_data_directory="testing_data/benchmark1"):
    """
    Benchmark the model by testing it on sorted data and calculating accuracy and
    detailed false positives per class.
    
    Args:
        test_data_directory: Directory containing sorted character folders
    """
    import os
    import numpy as np
    # NOTE: cnn_model, preprocess_image_for_cnn, and class_to_char must be defined globally
    # or passed as arguments for this function to run.
    
    if not os.path.exists(test_data_directory):
        print(f"Error: Test data directory '{test_data_directory}' not found!")
        return
    
    # Get all character folders
    character_folders = [f for f in os.listdir(test_data_directory) 
                         if os.path.isdir(os.path.join(test_data_directory, f))]
    
    if not character_folders:
        print("No character folders found in test directory!")
        return
    
    character_folders.sort()
    
    total_correct = 0
    total_images = 0
    overall_results = {}
    
    # Get all possible predicted characters from EMNIST (including lowercase)
    # This assumes class_to_char can return all possible characters
    all_possible_chars = set(character_folders)  # Start with folders that exist
    
    # We'll dynamically add predicted characters as we encounter them
    detailed_false_positives = {}
    
    print(f"Benchmarking model on {len(character_folders)} character classes...")
    print("="*60)
    
    for true_char in character_folders:
        folder_path = os.path.join(test_data_directory, true_char)
        
        # Get all image files in this character folder
        image_files = []
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                image_files.append(filename)
        
        if not image_files:
            print(f"No images found in folder '{true_char}'")
            continue
        
        print(f"\nTesting character '{true_char}' ({len(image_files)} images):")
        
        # Dictionary to count predictions for this character (True Positive + False Negatives)
        prediction_counts = {}
        correct_predictions = 0
        
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_cnn3(image_path)
            
            if preprocessed_image is not None:
                # Get prediction
                prediction_proba = cnn_model.predict(preprocessed_image, verbose=0)
                predicted_class_id = np.argmax(prediction_proba[0])
                predicted_char = class_to_char_emnist_bymerge(predicted_class_id)
                confidence = np.max(prediction_proba[0])
                
                # Count predictions
                prediction_counts[predicted_char] = prediction_counts.get(predicted_char, 0) + 1
                
                # Check if prediction is correct
                if predicted_char == true_char:
                    correct_predictions += 1
                else:
                    # Initialize the predicted_char entry if it doesn't exist
                    if predicted_char not in detailed_false_positives:
                        detailed_false_positives[predicted_char] = {}
                    
                    # Update the detailed False Positive tracker
                    if true_char in detailed_false_positives[predicted_char]:
                        detailed_false_positives[predicted_char][true_char] += 1
                    else:
                        detailed_false_positives[predicted_char][true_char] = 1
                
                total_images += 1
        
        # Calculate accuracy for this character
        folder_accuracy = (correct_predictions / len(image_files)) * 100 if image_files else 0
        total_correct += correct_predictions
        
        # Display results for this character
        print(f"Predictions for '{true_char}':")
        sorted_predictions = sorted(prediction_counts.items(), key=lambda x: x[1], reverse=True)
        for pred_char, count in sorted_predictions:
            percentage = (count / len(image_files)) * 100
            marker = "✓" if pred_char == true_char else "✗"
            print(f"  {marker} {pred_char}: {count} times ({percentage:.1f}%)")
        
        print(f"Accuracy for '{true_char}': {correct_predictions}/{len(image_files)} ({folder_accuracy:.1f}%)")
        
        # Store results
        overall_results[true_char] = {
            'total_images': len(image_files),
            'correct_predictions': correct_predictions,
            'accuracy': folder_accuracy,
            'predictions': prediction_counts
        }
    
    # Calculate and display overall accuracy
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%")
    
    # Display per-character summary including detailed FPs
    print("\nPer-character summary (Accuracy & Confusion):")
    print("Note: Only showing characters that exist in test folders")
    for char in character_folders:
        results = overall_results.get(char, {'accuracy': 0.0, 'correct_predictions': 0, 'total_images': 0})
        fp_details = detailed_false_positives.get(char, {})
        
        # Calculate total False Positives for this character (how many times it was predicted incorrectly)
        total_fp = sum(fp_details.values())
        
        # Format the detailed FP string (e.g., "4-Y, 2-U")
        fp_list = []
        if total_fp > 0:
            # Sort by count descending for readability
            sorted_fp = sorted(fp_details.items(), key=lambda x: x[1], reverse=True)
            fp_list = [f"{count}-{true_char}" for true_char, count in sorted_fp]
            
        fp_str = ", ".join(fp_list) if fp_list else "None"
        
        print(f"  {char}: Acc={results['accuracy']:.1f}% ({results['correct_predictions']}/{results['total_images']})")
        print(f"      FP ({total_fp}): {fp_str}")
        
        # Store False Positives in the results structure
        overall_results[char]['false_positives_total'] = total_fp
        overall_results[char]['false_positives_details'] = fp_details
    
    # Find best and worst performing characters
    if overall_results:
        best_char = max(overall_results.items(), key=lambda x: x[1]['accuracy'])
        worst_char = min(overall_results.items(), key=lambda x: x[1]['accuracy'])
        
        print(f"\nBest performing: '{best_char[0]}' ({best_char[1]['accuracy']:.1f}%)")
        print(f"Worst performing: '{worst_char[0]}' ({worst_char[1]['accuracy']:.1f}%)")
    
    # Show any lowercase predictions that were made but don't have folders
    print("\n" + "="*60)
    print("PREDICTED LOWERCASE CHARACTERS (no test folders):")
    print("="*60)
    lowercase_predictions = {char: fp_details for char, fp_details in detailed_false_positives.items() 
                            if char not in character_folders and char.islower()}
    
    if lowercase_predictions:
        for pred_char, true_char_dict in sorted(lowercase_predictions.items()):
            total = sum(true_char_dict.values())
            confused_with = ", ".join([f"{count}-{tc}" for tc, count in sorted(true_char_dict.items(), key=lambda x: x[1], reverse=True)])
            print(f"  '{pred_char}' was predicted {total} times")
            print(f"      Confused with: {confused_with}")
    else:
        print("  No lowercase predictions detected")
    
    return overall_results

### ALPHA

In [24]:
import cv2

def benchmark_alpha(test_data_directory="testing_data/benchmark1"):
    """
    Benchmark the alpha model by testing it on sorted data and calculating accuracy and
    detailed false positives per class. Only processes alphabetic character folders.
    
    Args:
        test_data_directory: Directory containing sorted character folders
    """
    import os
    import numpy as np
    # NOTE: cnn_model_alpha, preprocess_image_for_cnn, and class_to_char_alpha must be defined globally
    # or passed as arguments for this function to run.
    
    if not os.path.exists(test_data_directory):
        print(f"Error: Test data directory '{test_data_directory}' not found!")
        return
    
    # Get all character folders - ONLY ALPHA CHARACTERS
    all_folders = [f for f in os.listdir(test_data_directory) 
                   if os.path.isdir(os.path.join(test_data_directory, f))]
    
    # Filter to only alphabetic characters
    character_folders = [f for f in all_folders if f.isalpha() and len(f) == 1]
    
    if not character_folders:
        print("No alpha character folders found in test directory!")
        return
    
    character_folders.sort()
    
    total_correct = 0
    total_images = 0
    overall_results = {}
    
    # NEW: Initialize a dictionary to track False Positives in detail.
    # Structure: {predicted_char: {true_char: count, ...}, ...}
    detailed_false_positives = {char: {} for char in character_folders}
    
    print(f"Benchmarking alpha model on {len(character_folders)} alphabetic character classes...")
    print("="*60)
    
    for true_char in character_folders:
        folder_path = os.path.join(test_data_directory, true_char)
        
        # Get all image files in this character folder
        image_files = []
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                image_files.append(filename)
        
        if not image_files:
            print(f"No images found in folder '{true_char}'")
            continue
        
        print(f"\nTesting character '{true_char}' ({len(image_files)} images):")
        
        # Dictionary to count predictions for this character (True Positive + False Negatives)
        prediction_counts = {}
        correct_predictions = 0
        
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_cnn(image_path)
            
            if preprocessed_image is not None:
                # Get prediction using alpha model                
                #print(f"Preprocessed image shape: {preprocessed_image.shape}")

                prediction_proba = cnn_model.predict(preprocessed_image, verbose=0)
                predicted_class_id = np.argmax(prediction_proba[0])
                predicted_char = class_to_char_alpha(predicted_class_id)
                confidence = np.max(prediction_proba[0])
                
                # Count predictions
                prediction_counts[predicted_char] = prediction_counts.get(predicted_char, 0) + 1
                
                # Check if prediction is correct
                if predicted_char == true_char:
                    correct_predictions += 1
                else:
                    # NEW: Update the detailed False Positive tracker
                    # The model predicted 'predicted_char' when the true label was 'true_char'
                    if true_char in detailed_false_positives[predicted_char]:
                        detailed_false_positives[predicted_char][true_char] += 1
                    else:
                        detailed_false_positives[predicted_char][true_char] = 1
                
                total_images += 1
        
        # Calculate accuracy for this character
        folder_accuracy = (correct_predictions / len(image_files)) * 100 if image_files else 0
        total_correct += correct_predictions
        
        # Display results for this character
        print(f"Predictions for '{true_char}':")
        sorted_predictions = sorted(prediction_counts.items(), key=lambda x: x[1], reverse=True)
        for pred_char, count in sorted_predictions:
            percentage = (count / len(image_files)) * 100
            marker = "✓" if pred_char == true_char else "✗"
            print(f"  {marker} {pred_char}: {count} times ({percentage:.1f}%)")
        
        print(f"Accuracy for '{true_char}': {correct_predictions}/{len(image_files)} ({folder_accuracy:.1f}%)")
        
        # Store results
        overall_results[true_char] = {
            'total_images': len(image_files),
            'correct_predictions': correct_predictions,
            'accuracy': folder_accuracy,
            'predictions': prediction_counts
        }
    
    # Calculate and display overall accuracy
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY (ALPHA MODEL)")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%")
    
    # Display per-character summary including detailed FPs
    print("\nPer-character summary (Accuracy & Confusion):")
    for char in character_folders:
        results = overall_results.get(char, {'accuracy': 0.0, 'correct_predictions': 0, 'total_images': 0})
        fp_details = detailed_false_positives.get(char, {})
        
        # Calculate total False Positives for this character (how many times it was predicted incorrectly)
        total_fp = sum(fp_details.values())
        
        # Format the detailed FP string (e.g., "4-Y, 2-U")
        fp_list = []
        if total_fp > 0:
            # Sort by count descending for readability
            sorted_fp = sorted(fp_details.items(), key=lambda x: x[1], reverse=True)
            fp_list = [f"{count}-{true_char}" for true_char, count in sorted_fp]
            
        fp_str = ", ".join(fp_list) if fp_list else "None"
        
        print(f"  {char}: Acc={results['accuracy']:.1f}% ({results['correct_predictions']}/{results['total_images']})")
        print(f"      FP ({total_fp}): {fp_str}")
        
        # Store False Positives in the results structure
        overall_results[char]['false_positives_total'] = total_fp
        overall_results[char]['false_positives_details'] = fp_details
    
    # Find best and worst performing characters
    if overall_results:
        best_char = max(overall_results.items(), key=lambda x: x[1]['accuracy'])
        worst_char = min(overall_results.items(), key=lambda x: x[1]['accuracy'])
        
        print(f"\nBest performing: '{best_char[0]}' ({best_char[1]['accuracy']:.1f}%)")
        print(f"Worst performing: '{worst_char[0]}' ({worst_char[1]['accuracy']:.1f}%)")
    
    overall_accuracy = (total_correct / total_images) * 100 if total_images > 0 else 0
    
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY (ALPHA MODEL)")
    print("="*60)
    print(f"Total images tested: {total_images}")
    print(f"Total correct predictions: {total_correct}")
    print(f"Overall accuracy: {overall_accuracy:.2f}%") # <-- THIS IS THE FINAL METRIC
    

In [ ]:
base_dir_cnn = "models/cnn/"
alpha = "alpha_cnn.h5"
alpha2 = "alpha_recognition_cnn2.h5"
alpha3 = "alpha_recognition_cnn3.h5"
alpha4 = "alpha_test_50_cnn.keras"
alphanum = "alphanum_cnn.h5"
alphanum2 = "alphanum_recognition_cnn.h5"
alphanum3 = "alphanum_regularized_cnn.h5"
alphanum4 = "alpha_recognition_cnn2.h5"
emnist = "emnist_cnn.h5"
emnist_az = ""
emnist_az_reduced = "eminst_a-z_reduced_cnn.h5"


cnn_model = tf.keras.models.load_model(base_dir_cnn + alpha4)

In [26]:
benchmark_alpha()

Benchmarking alpha model on 26 alphabetic character classes...

Testing character 'A' (174 images):
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1)
Preprocessed image shape: (1, 28, 28, 1

KeyboardInterrupt: 